## torchvision의 transform으로 이미지 정규화하기(평균, 표준편차를 계산하여 적용)
출처: <a href="https://teddylee777.github.io/pytorch/torchvision-transform">https://teddylee777.github.io/pytorch/torchvision-transform</a>

이미지 정규화를 진행하는 대표적인 이유 중 하나는 오차역전파(backpropagation)시, 그라디언트(Gradient) 계산을 수행하게 되는데, 데이터가 유사한 범위를 가지도록 하기 위함
정규화를 어떻게 수행하는가에 따라서 모델의 학습결과는 달라질 수 있음

In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms

In [2]:
# 0~1의 범위를 가지도록 정규화
transform = transforms.Compose([transforms.ToTensor(),])

# datasets의 CIFAR10 데이터셋 로드 (train 데이터셋)
train = datasets.CIFAR10(root='data', 
                         train=True, 
                         download=True, 
                         # transform 지정
                         transform=transform                
                        )

# datasets의 CIFAR10 데이터셋 로드 (test 데이터셋)
test = datasets.CIFAR10(root='data', 
                        train=False, 
                        download=True, 
                        # transform 지정
                        transform=transform
                       )                        

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# 이미지의 RGB 채널별 통계량 확인 함수
def print_stats(dataset):
    imgs = np.array([img.numpy() for img, _ in dataset])
    print(f'shape: {imgs.shape}')
    
    # axis=(2,3) => 32x32 이미지 데이터 중 에서 연산(최소, 최대, 평균, 표준편차) 수행
    min_r = np.min(imgs, axis=(2, 3))[:, 0].min()
    min_g = np.min(imgs, axis=(2, 3))[:, 1].min()
    min_b = np.min(imgs, axis=(2, 3))[:, 2].min()

    max_r = np.max(imgs, axis=(2, 3))[:, 0].max()
    max_g = np.max(imgs, axis=(2, 3))[:, 1].max()
    max_b = np.max(imgs, axis=(2, 3))[:, 2].max()

    mean_r = np.mean(imgs, axis=(2, 3))[:, 0].mean()
    mean_g = np.mean(imgs, axis=(2, 3))[:, 1].mean()
    mean_b = np.mean(imgs, axis=(2, 3))[:, 2].mean()

    std_r = np.std(imgs, axis=(2, 3))[:, 0].std()
    std_g = np.std(imgs, axis=(2, 3))[:, 1].std()
    std_b = np.std(imgs, axis=(2, 3))[:, 2].std()
    
    print(f'min: {min_r, min_g, min_b}')
    print(f'max: {max_r, max_g, max_b}')
    print(f'mean: {mean_r, mean_g, mean_b}')
    print(f'std: {std_r, std_g, std_b}')

In [4]:
# transforms.ToTensor()만 적용한 경우, 모든 이미지의 픽셀 값이 0~1의 범위로 변환됨
print_stats(train)
print('==='*10)
print_stats(test)

shape: (50000, 3, 32, 32)
min: (0.0, 0.0, 0.0)
max: (1.0, 1.0, 1.0)
mean: (0.49139965, 0.48215845, 0.4465309)
std: (0.060528398, 0.061124973, 0.06764512)
shape: (10000, 3, 32, 32)
min: (0.0, 0.0, 0.0)
max: (1.0, 1.0, 1.0)
mean: (0.49421427, 0.48513138, 0.45040908)
std: (0.06047972, 0.06123986, 0.06758436)


### transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 적용

transofrms.Normalize()는 각 채널별 평균(mean)을 뺀 뒤 표준편차(std)로 나누어 정규화를 진행
transofrms.Normalize((R채널 평균, G채널 평균, B채널 평균), (R채널 표준편차, G채널 표준편차, B채널 표준편차))

변환 후 결과 = (픽셀 값 - R채널 평균) / (R채널 표준편차)
transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 적용한 경우

transforms.ToTensor()가 이미지 픽셀 값의 범위를 0 ~ 1 로 조정했으므로,
최소값(=-1)은 (0 - 0.5) / 0.5 = -1, 최대값(=1) 은 (1 - 0.5) / 0.5 = 1 로 조정
결국, 위의 예시를 적용한 결과는 -1 ~ 1 범위로 변환됨

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    # -1 ~ 1 사이의 범위를 가지도록 정규화
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# datasets의 CIFAR10 데이터셋 로드 (train 데이터셋)
train = datasets.CIFAR10(root='data', 
                         train=True, 
                         download=True, 
                         transform=transform                
                        )

# datasets의 CIFAR10 데이터셋 로드 (test 데이터셋)
test = datasets.CIFAR10(root='data', 
                        train=False, 
                        download=True, 
                        transform=transform
                       )     
                       
print_stats(train)
print('==='*10)
print_stats(test)

Files already downloaded and verified
Files already downloaded and verified
shape: (50000, 3, 32, 32)
min: (-1.0, -1.0, -1.0)
max: (1.0, 1.0, 1.0)
mean: (-0.017200625, -0.035683163, -0.10693816)
std: (0.121056795, 0.122249946, 0.13529024)
shape: (10000, 3, 32, 32)
min: (-1.0, -1.0, -1.0)
max: (1.0, 1.0, 1.0)
mean: (-0.011571422, -0.029737204, -0.0991818)
std: (0.12095944, 0.12247972, 0.13516872)
